In [1]:

import json
import os

import gspread
import pandas as pd

def get_worksheet(ws_name):
    credentials_dict_str = os.environ["GOOGLE_CREDENTIALS"]
    credentials_dict = json.loads(credentials_dict_str)
    g_credentials = gspread.service_account_from_dict(credentials_dict)
    g_sheet = g_credentials.open("Korea - Users stats")
    worksheet = g_sheet.worksheet(ws_name)

    return worksheet


def update_worksheet(dataframe, worksheet):
    dataframe = dataframe.fillna("")
    df_list = [dataframe.columns.values.tolist()]
    df_list += dataframe.values.tolist()
    worksheet.update(df_list, value_input_option="USER_ENTERED")



In [2]:

vocab_g_ws = get_worksheet("Freezpmark")
df = pd.DataFrame(vocab_g_ws.get_all_records())

from datetime import datetime
from statistics import fmean

def scoring(df):
    d = {}
    d["Score"] = df["Session number"].sum()
    return pd.Series(d, index=["Score"])

# df["Date"] - pd.to_datetime("now")
# df.groupby("Word").apply(scoring)

df.sort_values(["Word", "Date"], ascending=[True, False], inplace=True)



In [3]:
# Getting the current date and time
dt = datetime.now()
dt2 = "2023-01-13 07:49:19"
datetime_object = datetime.strptime(dt2, "%Y-%m-%d %H:%M:%S")
ts = (dt - datetime_object).days
ts

10

In [23]:
distr = (1, 0.8, 0.64, 0.512, 0.4096, 0.32768, 0.262144, 0.209715, 0.167772, 0.1342176)
scores = {i: j for i, j in enumerate(distr)}

datetime_now = datetime.now()

knowledge = []
word_scores = {}
new_word = ""
new_word_counter = 1
new_word_score = 0
for row in df.itertuples():
    if new_word_counter >= 10:
        continue  # needs opt -> use pd
    if new_word == row.Word:
        new_word_counter += 1
    else:
        if new_word:
            # empty_fill = sum(4*x for x in distr[new_word_counter+1::])
            empty_fill = (fmean(knowledge) * (10-new_word_counter))  # mean
            word_scores[new_word] = new_word_score + empty_fill
            knowledge = []
            new_word_score = 0
            new_word_counter = 1
        new_word = row.Word

    if row.Knowledge == "✅":
        knowledge_multiplier = 1
    elif row.Knowledge == "⏭️":
        knowledge_multiplier = 2
    elif row.Knowledge == "❌":
        knowledge_multiplier = 4
    # new_word_score.append(f"{knowledge_multiplier}*{scores[new_word_counter]}")
    
    new_word_score += knowledge_multiplier * scores[new_word_counter]
    knowledge.append(knowledge_multiplier)

    row_datetime = datetime.strptime(row.Date, "%Y-%m-%d %H:%M:%S")
    now_datetime = datetime_now
    days_diff = (now_datetime - row_datetime).days
    new_word_score += days_diff * 0.01

    if row.Word == "열쇠" or row.Word == "뭐1":
        print(f"{row.Word}: {knowledge_multiplier}*{scores[new_word_counter]};; {new_word_score}")


sorted_word_score = list(sorted(word_scores.items(), key=lambda item: item[1], reverse=True))
sorted_word_score


뭐1: 1*0.8;; 0.9
뭐1: 1*0.64;; 1.6500000000000001
열쇠: 2*0.8;; 1.7000000000000002
열쇠: 1*0.64;; 2.45


[('필통', 23.56),
 ('끄다', 20.761333333333337),
 ('여권', 20.761333333333337),
 ('간호사', 16.5024),
 ('열쇠', 14.45),
 ('사전2', 14.280000000000001),
 ('열다', 14.280000000000001),
 ('읽다', 14.280000000000001),
 ('켜다', 14.280000000000001),
 ('동생', 12.097333333333331),
 ('의사1', 12.097333333333331),
 ('가다1', 9.9),
 ('가르치다', 9.9),
 ('가방', 9.9),
 ('거울', 9.9),
 ('것', 9.9),
 ('교실', 9.9),
 ('그것', 9.9),
 ('깨끗하다', 9.9),
 ('꽃', 9.9),
 ('나쁘다', 9.9),
 ('날씨', 9.9),
 ('닫다', 9.9),
 ('도서관', 9.9),
 ('마시다', 9.9),
 ('많다', 9.9),
 ('맛없다', 9.9),
 ('맛있다', 9.9),
 ('머리', 9.9),
 ('멋있다', 9.9),
 ('모자', 9.9),
 ('무엇', 9.9),
 ('문', 9.9),
 ('물', 9.9),
 ('바나나', 9.9),
 ('바쁘다', 9.9),
 ('밥', 9.9),
 ('배우다', 9.9),
 ('보내다', 9.9),
 ('보다1', 9.9),
 ('볼펜', 9.9),
 ('불', 9.9),
 ('불고기', 9.9),
 ('빵', 9.9),
 ('사과1', 9.9),
 ('사다', 9.9),
 ('서다', 9.9),
 ('쉬다1', 9.9),
 ('싸다1', 9.9),
 ('쓰다2', 9.9),
 ('아메리카노', 9.9),
 ('아프다', 9.9),
 ('안경', 9.9),
 ('앉다', 9.9),
 ('에어컨', 9.9),
 ('영화', 9.9),
 ('예쁘다', 9.9),
 ('오다1', 9.9),
 ('오렌지', 9.9),
 ('우산', 9.9),
 ('우유',

In [26]:
import random

nl = []
size = 10
for i in range(0, len(sorted_word_score), size):
    if i > 4:
        nl += sorted_word_score[i::]
        break
    subset = sorted_word_score[i:i+size]
    random.shuffle(subset)
    nl += subset
nl[::-1]


[('좋아하다', 9.242),
 ('씨1', 9.252),
 ('멕시코', 9.252),
 ('가수', 9.252),
 ('회사원', 9.64),
 ('한국어', 9.64),
 ('한국', 9.64),
 ('학생', 9.64),
 ('필리핀', 9.64),
 ('프랑스', 9.64),
 ('커피', 9.64),
 ('카메라', 9.64),
 ('처음', 9.64),
 ('책', 9.64),
 ('직업', 9.64),
 ('지우개', 9.64),
 ('중국', 9.64),
 ('주부', 9.64),
 ('주다1', 9.64),
 ('좋다', 9.64),
 ('저것', 9.64),
 ('저1', 9.64),
 ('일본', 9.64),
 ('이름', 9.64),
 ('이것', 9.64),
 ('은행원', 9.64),
 ('유학생', 9.64),
 ('요리사', 9.64),
 ('옷', 9.64),
 ('영어', 9.64),
 ('영국', 9.64),
 ('연필', 9.64),
 ('여자', 9.64),
 ('어느', 9.64),
 ('아니다', 9.64),
 ('시계', 9.64),
 ('사람', 9.64),
 ('비싸다', 9.64),
 ('베트남', 9.64),
 ('배우', 9.64),
 ('미용사', 9.64),
 ('미국', 9.64),
 ('먹다1', 9.64),
 ('디자이너', 9.64),
 ('누구', 9.64),
 ('네1', 9.64),
 ('남자', 9.64),
 ('나라', 9.64),
 ('기다리다', 9.64),
 ('공책', 9.64),
 ('공부하다', 9.64),
 ('공부', 9.64),
 ('뭐1', 9.65),
 ('휴대폰', 9.9),
 ('화장품', 9.9),
 ('편지', 9.9),
 ('텔레비전', 9.9),
 ('크다', 9.9),
 ('컴퓨터', 9.9),
 ('칠판', 9.9),
 ('청소를 하다', 9.9),
 ('책상', 9.9),
 ('창문', 9.9),
 ('지금1', 9.9),
 ('지갑', 9.9),
 

In [32]:
a = nl[-1][0]
a

'좋아하다'